In [17]:
from choquet_integral import *
import numpy as np
import cv2
import itertools
import math
import inspect
import matplotlib.pyplot as plt
import csv
import sklearn.metrics as metrics
from scipy.stats import entropy
import random
import xai_indices as xai
import pyemd
from sklearn import datasets
from emd_clustertend import vat, ivat
from sklearn.preprocessing import scale
import scipy.io as sio

In [2]:
def random_node_fm(n):
    ch = ChoquetIntegral()
    ch.type='quad'
    ch.fm = {}
    ch.fm[str(np.arange(1,n+1))] = 1
    ch.fm['[]'] = 0
    ch.N = n
    ch.M = n
    numkeys = len(ch.get_keys_index())
    keys = list(ch.get_keys_index().keys())
    keys.append('[]')
    s = [0] * (numkeys + 1)
    s[-1] = 1
    s[-2] = 1
    done = False
    while not done:
        randindex = random.randrange(0,numkeys)
        if s[randindex] == 0:
            s[randindex] = 1
            if keys[randindex] != '[]':
                compare_key = [int(s) for s in keys[randindex][1:-1].split() if s.isdigit()]

            else:
                compare_key = []

            maxi = 0
            max_index = 0
            mini = 1
            min_index = 0
            for i,key in enumerate(keys):
                if s[i] == 1 and i != randindex:
                    if key != '[]':
                        stripped = [int(s) for s in key[1:-1].split() if s.isdigit()]
                    else:
                        stripped = []

                    if is_subset(stripped,compare_key) and s[i] == 1 and ch.fm[key] >= maxi:
                        maxi = ch.fm[key]
                        max_index = i
                    if is_subset(compare_key,stripped) and s[i] == 1 and ch.fm[key] <= mini:

                        mini = ch.fm[key]
                        min_index = i


            rb = ch.fm[keys[max_index]]
            ru = ch.fm[keys[min_index]]
            g = random.uniform(rb,ru)
            ch.fm[keys[randindex]] = g

        else:
            pass
        if min(s) == 1:
            done = True
    return ch

# a is subset of b
def is_subset(a,b):
    if len(a) == 0:
        return True
    if len(b) == 0 and len(a) != 0:
        return False
    for val in a:
        if val not in b:
            return False
    else:
        return True
    
def sample_with_noise(ch,data,mean,var):
    labels = np.zeros(data.shape[0])
    for i,point in enumerate(data):
        labels[i] = max(min(ch.chi_quad(point) + random.gauss(mean,var),1),0)
    return labels

def gen_datapoints(m,n):
    points = []
    for i in range(n):
        point = []
        for j in range(m):
            point.append(random.random())
        points.append(point)
    return np.asarray(points)

def percentage_walks_observed(walks):
    seen = 0
    total = 0
    for key in walks.keys():
        if walks[key] > 0:
            seen = seen + 1
        total = total+1
    return seen/total

def read_crisp(fname):
    part = []
    with open(fname) as csvfile:
        reader = csv.reader(csvfile,delimiter=',')
        for row in reader:
            part.append(row)
    return np.asarray(part,dtype=int)
crisp_part = read_crisp("crisp.csv")

#input: crisp partition matrix, reordering from odm to dm, odm itself
def center_from_crisp(crisp,reordering,odm):
    n = crisp.shape[0]
    row = 0
    num_in_cluster = 0
    cluster_centers = []
    for i in range(n):
        if crisp[row][i] == 1:
            num_in_cluster = num_in_cluster + 1
            
        else:
#             print("new cluster at", i)
            mini = np.inf
            mini_row = row
            for j in range(row,row+num_in_cluster):
                row_sum = np.sum(odm[j][i-num_in_cluster:i])
                if row_sum < mini:
                    mini = row_sum
                    mini_row = j
            cluster_centers.append(reorder[mini_row][0][0])   #the center is row w/ minimum difference, put back in unordered
            row = row+num_in_cluster
            num_in_cluster = 1
            
    mini = np.inf
    mini_row = row
    for j in range(row,row+num_in_cluster):
        row_sum = np.sum(odm[j][i-num_in_cluster+1:i+1])
#         print(row_sum, "sum of row", j)
        if row_sum < mini:
            mini = row_sum
            mini_row = j
    cluster_centers.append(reorder[mini_row][0][0])
    return cluster_centers

def generate_walk_diffs(chi):
    n = chi.M
    initial = []
    for i in range(n):
        initial.append(i+1)

    index_permutations = itertools.permutations(initial,n)
    walks = np.ndarray((math.factorial(n),n),dtype=int)
    walk_diffs = np.ndarray((math.factorial(n),n),dtype=float)
    for i,val in enumerate(index_permutations):
        walks[i] = np.asarray(val)

        c_build = np.ndarray(n)
        for j in range(len(walks[i])):
            if j == 0:
                c_build[j] = chi.fm[str(walks[i][0:1])]
            else:
                c_build[j] = (chi.fm[str(np.sort(walks[i][0:j+1]))]) - (chi.fm[str(np.sort(walks[i][0:j]))])

        walk_diffs[i] = c_build
    return walk_diffs

In [ ]:

def gen_seen_walk_diffs(walk_visitation,ch):
    seen_walks = []
    for walk in walk_visitation.keys():
        if walk_visitation[walk] > 0:
            seen_walks.append(walk)
    
    n = len(seen_walks[0])
    walk_diffs = np.ndarray((len(seen_walks),n),dtype=float)
    for i,walk in enumerate(seen_walks):
        walk = list(walk)
        c_build = np.ndarray(n)
        for j in range(len(walk)):
            if j == 0:
                c_build[j] = ch.fm[str(walk[0:1])]
            else:
                c_build[j] = ch.fm[str(np.sort(walk[0:j+1]))] - ch.fm[str(np.sort(walk[0:j]))]
        walk_diffs[i] = c_build
    return walk_diffs
        
    
(walks,_) = xai.walk_visitation(data.T)
seen_diffs = gen_seen_walk_diffs(walks,noisy_split_owa)
all_diffs = noisy_split_owa.generate_walk_diffs()

ivat(all_diffs,figure_size=(5,5),euclidean=True)
ivat(all_diffs,figure_size=(5,5))

In [ ]:
custom = ChoquetIntegral()
custom.N = 4
customfm = {}
customfm[str(np.asarray([1]))] = .9
customfm[str(np.asarray([2]))] = 0
customfm[str(np.asarray([3]))] = 0
customfm[str(np.asarray([4]))] = 0
customfm[str(np.asarray([1,2]))] = .9
customfm[str(np.asarray([1,3]))] = .9
customfm[str(np.asarray([1,4]))] = .9
customfm[str(np.asarray([2,3]))] = 0
customfm[str(np.asarray([2,4]))] = 0
customfm[str(np.asarray([3,4]))] = 0
customfm[str(np.asarray([1,2,3]))] = .9
customfm[str(np.asarray([1,2,4]))] = .9
customfm[str(np.asarray([1,3,4]))] = .9
customfm[str(np.asarray([2,3,4]))] = 0
customfm[str(np.asarray([1,2,3,4]))] = 1

custom.fm = customfm
custom.type='quad'
print(custom.fm)


In [ ]:
split_owa = ChoquetIntegral()
split_owa.N = 5
split_fm = {}
split_fm[str(np.asarray([1]))] = .1
split_fm[str(np.asarray([2]))] = .1
split_fm[str(np.asarray([3]))] = .1
split_fm[str(np.asarray([4]))] = .1
split_fm[str(np.asarray([5]))] = .1
split_fm[str(np.asarray([1,2]))] = .2
split_fm[str(np.asarray([1,3]))] = .2
split_fm[str(np.asarray([1,4]))] = .2
split_fm[str(np.asarray([1,5]))] = .2
split_fm[str(np.asarray([2,3]))] = .2
split_fm[str(np.asarray([2,4]))] = .4
split_fm[str(np.asarray([2,5]))] = .4
split_fm[str(np.asarray([3,4]))] = .4
split_fm[str(np.asarray([3,5]))] = .4
split_fm[str(np.asarray([4,5]))] = .4
split_fm[str(np.asarray([1,2,3]))] = .6
split_fm[str(np.asarray([1,2,4]))] = .6
split_fm[str(np.asarray([1,2,5]))] = .6
split_fm[str(np.asarray([1,3,4]))] = .6
split_fm[str(np.asarray([1,3,5]))] = .6
split_fm[str(np.asarray([1,4,5]))] = .6
split_fm[str(np.asarray([2,3,4]))] = .6
split_fm[str(np.asarray([2,3,5]))] = .6
split_fm[str(np.asarray([2,4,5]))] = .6
split_fm[str(np.asarray([3,4,5]))] = .6
split_fm[str(np.asarray([1,2,3,4]))] = .6
split_fm[str(np.asarray([1,2,3,5]))] = .6
split_fm[str(np.asarray([1,2,4,5]))] = .6
split_fm[str(np.asarray([1,3,4,5]))] = .9
split_fm[str(np.asarray([2,3,4,5]))] = .9
split_fm[str(np.asarray([1,2,3,4,5]))] = 1
split_owa.fm = split_fm
split_owa.type ='quad'

In [ ]:
max_min = ChoquetIntegral()
max_min.N = 5
max_min.M = 5
max_min_fm = {}
max_min_fm['[]'] = 0
max_min_fm[str(np.asarray([1]))] = .1
max_min_fm[str(np.asarray([2]))] = 0
max_min_fm[str(np.asarray([3]))] = 0
max_min_fm[str(np.asarray([4]))] = 0
max_min_fm[str(np.asarray([5]))] = 0
max_min_fm[str(np.asarray([1,2]))] = .1
max_min_fm[str(np.asarray([1,3]))] = .1
max_min_fm[str(np.asarray([1,4]))] = .1
max_min_fm[str(np.asarray([1,5]))] = .1
max_min_fm[str(np.asarray([2,3]))] = 0
max_min_fm[str(np.asarray([2,4]))] = 0
max_min_fm[str(np.asarray([2,5]))] = 0
max_min_fm[str(np.asarray([3,4]))] = 0
max_min_fm[str(np.asarray([3,5]))] = 0
max_min_fm[str(np.asarray([4,5]))] = 0
max_min_fm[str(np.asarray([1,2,3]))] = .1
max_min_fm[str(np.asarray([1,2,4]))] = .1
max_min_fm[str(np.asarray([1,2,5]))] = .1
max_min_fm[str(np.asarray([1,3,4]))] = .1
max_min_fm[str(np.asarray([1,3,5]))] = .1
max_min_fm[str(np.asarray([1,4,5]))] = .1
max_min_fm[str(np.asarray([2,3,4]))] = 0
max_min_fm[str(np.asarray([2,3,5]))] = 0
max_min_fm[str(np.asarray([2,4,5]))] = 0
max_min_fm[str(np.asarray([3,4,5]))] = 0
max_min_fm[str(np.asarray([1,2,3,4]))] = .1001
max_min_fm[str(np.asarray([1,2,3,5]))] = .1001
max_min_fm[str(np.asarray([1,2,4,5]))] = .1001
max_min_fm[str(np.asarray([1,3,4,5]))] = .1001
max_min_fm[str(np.asarray([2,3,4,5]))] = 0
max_min_fm[str(np.asarray([1,2,3,4,5]))] = 1
max_min.fm = max_min_fm
max_min.type ='quad'

In [ ]:
max_min.fm
max_min_diffs = max_min.generate_walk_diffs()
ivat(max_min_diffs)
ivat(max_min_diffs,euclidean=True)

In [ ]:
data = gen_datapoints(5,300)
noisy_labels = sample_with_noise(split_owa,data,0,0)

noisy_split_owa = ChoquetIntegral()
noisy_split_owa.train_chi(data.T,noisy_labels)

(walks,_) = xai.walk_visitation(data.T)
# print(walks)
print(percentage_walks_observed(walks))

owa_diffs = split_owa.generate_walk_diffs()
ivat(owa_diffs,figure_size=(5,5))
noisy_owa_diffs = noisy_split_owa.generate_walk_diffs()
odm = ivat(noisy_owa_diffs,figure_size=(5,5),return_odm=True)

In [ ]:
plt.imshow(odm,cmap="gray")
plt.savefig("split_owa_no_error.png")

In [ ]:
data = gen_datapoints(4,200)
noisy_labels = sample_with_noise(split_owa,data,0,0)

noisy_chi = ChoquetIntegral()
noisy_chi.train_chi(data.T,noisy_labels)

xai.walk_visitation(data.T)

In [ ]:
(walks,_) = xai.walk_visitation(data.T)


percentage_walks_observed(walks)
# print(walks.keys())

In [ ]:

ch = random_node_fm(6)

diffs = ch.generate_walk_diffs()
odm = ivat(diffs,return_odm=True,figure_size=(5,5),euclidean=False)

# odm,reorder = ivat(diffs,return_odm=True,euclidean=True)

In [ ]:
plt.imshow(odm,cmap='gray')
plt.savefig("6var_struct")
# print(center_from_crisp(crisp_part,reorder,odm))
# ch.generate_walk_diffs()

In [ ]:
img = cv2.imread("split_owa_noisy_.025.png")
y,x = img.shape[0], img.shape[1]
img = img[10:y-10,75:x-75]
cv2.imwrite("split_owa_noisy_.025.png",img)

In [ ]:
with open('odm.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in odm:
        writer.writerow(row)

In [3]:
keys = []
for g in range(1,2**7):
    key = []
    for i,val in enumerate(format(g,'b')[::-1]):
        if val == '1':
            key.append(i+1)
    keys.append(np.asarray(key))


In [4]:
learner_fm = {}

with open('FMs/fmvars_shared_0_0.txt', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for i,row in enumerate(reader):
        learner_fm[str(keys[i])] = float(row[0])


In [5]:
learner_ch = ChoquetIntegral()
learner_ch.N = 7
learner_ch.M = 7
learner_ch.type = 'quad'
learner_ch.fm = learner_fm
learner_diffs = learner_ch.generate_walk_diffs()
print("done with diffs")
# ivat(learner_diffs)

done with diffs


In [14]:
with open("big_diffs.csv",'w') as csvfile:
    writer = csv.writer(csvfile,delimiter=',')
    for i in range(diff_mat.shape[0]):
        writer.writerow(diff_mat[i])

In [21]:
# diff_mat = metrics.pairwise_distances(learner_diffs)
mat_dict = {}
mat_dict["diffs"] = diff_mat

In [22]:
sio.savemat("diffs.mat",mat_dict)